### Transformação de texto em áudio (text to speech)

In [ ]:
from gtts import gTTS

text_to_say = "Hello WORLD!."

language = "en"

gtts_object = gTTS(text = text_to_say, 
                  lang = language,
                  slow = False,
                  tld='co.uk')

gtts_object.save("C:/_ML/DIO_PLN/audio/en.wav")

from IPython.display import Audio

Audio("C:/_ML/DIO_PLN/audio/en.wav")

In [ ]:
portuguese_text = "Olá MUNDO!!!"

portuguese_language = "pt-br"

portuguese_gtts_object = gTTS(text = portuguese_text,
                              lang = portuguese_language,
                              slow = False,
                              tld='pt')

portuguese_gtts_object.save("C:/_ML/DIO_PLN/audio/pt.wav")

Audio("C:/_ML/DIO_PLN/audio/pt.wav")

### Transformação de áudio em texto (speech to text)

In [ ]:
#import section
import speech_recognition as sr
from gtts import gTTS
import os
from datetime import datetime
import playsound
import pyjokes
import wikipedia
import webbrowser
import winshell
from pygame import mixer
import time

# Initialize pygame mixer once
mixer.init()

#get mic audio
def get_audio():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        r.pause_threshold = 1
        r.adjust_for_ambient_noise(source, duration=1)
        audio = r.listen(source)
        said = ""
        try:
            said = r.recognize_google(audio)
            print(said)
        except sr.UnknownValueError:
            speak("Sorry, I did not get that.")
        except sr.RequestError:
            speak("Sorry, the service is not available")
    return said.lower()

#speak converted audio to text
def speak(text):
    # Create unique filename with timestamp
    timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
    audio_dir = os.path.abspath("audio")
    filename = os.path.join(audio_dir, f"voice_{timestamp}.mp3")
    
    # Ensure directory exists
    os.makedirs(audio_dir, exist_ok=True)
    
    # Generate TTS
    tts = gTTS(text=text, lang='en')
    tts.save(filename)
    
    # Play using pygame mixer to avoid playsound issues
    mixer.music.load(filename)
    mixer.music.play()
    
    # Wait for playback to finish
    while mixer.music.get_busy():
        time.sleep(0.1)
    
    # Cleanup file after playback
    try:
        os.remove(filename)
    except Exception as e:
        print(f"Error removing file: {e}")

#function to respond to commands
def respond(text):
    print("Text from get audio " + text)
    if 'youtube' in text:
        speak("What do you want to search for?")
        keyword = get_audio()
        if keyword != '':
            url = f"https://www.youtube.com/results?search_query={keyword}"
            webbrowser.get().open(url)
            speak(f"Here is what I have found for {keyword} on youtube")
    elif 'search' in text:
        speak("What do you want to search for?")
        query = get_audio()
        if query != '':
            try:
                result = wikipedia.summary(query, sentences=3)
                speak("According to wikipedia")
                print(result)
                speak(result)
            except wikipedia.exceptions.DisambiguationError as e:
                speak("There are multiple matches. Please be more specific.")
            except wikipedia.exceptions.PageError:
                speak("Sorry, I couldn't find any information on that.")
    elif 'joke' in text:
        speak(pyjokes.get_joke())
    elif 'empty recycle bin' in text:
        winshell.recycle_bin().empty(confirm=False, show_progress=False, sound=True)
        speak("Recycle bin emptied")
    elif 'what time' in text:
        strTime = datetime.today().strftime("%H:%M %p")
        print(strTime)
        speak(strTime)
    elif 'play music' in text or 'play song' in text:
        speak("Now playing...")
        music_dir = "C:\\Users\\Juanito\\Downloads\\Music\\"
        if os.path.exists(music_dir):
            songs = os.listdir(music_dir)
            if songs:
                playmusic(os.path.join(music_dir, songs[0]))
            else:
                speak("No songs found in the directory.")
        else:
            speak("Music directory not found.")
    elif 'stop music' in text:
        speak("Stopping playback.")
        stopmusic()
    elif 'exit' in text:
        speak("Goodbye, till next time")
        exit()

#play music
def playmusic(song):
    mixer.music.load(song)
    mixer.music.play()

#stop music
def stopmusic():
    mixer.music.stop()

# Main loop
while True:
    print("I am listening...")
    text = get_audio()
    respond(text)